<a href="https://colab.research.google.com/github/bcury/Machine-Learning-Projects/blob/main/ML_ARIMA_WebScrapping_Feminicidio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prever Feminicidios | dados do Governo de SP

In [ ]:
pip install pmdarima

     |████████████████████████████████| 1.4 MB 21.3 MB/s 
     |████████████████████████████████| 9.8 MB 25.6 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import plotly.graph_objects as Dash
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
site = 'http://www.ssp.sp.gov.br/Estatistica/ViolenciaMulher.aspx'

In [3]:
pagina = requests.get(site)

In [4]:
coleta = BeautifulSoup(pagina.text, 'html.parser')

In [5]:
print(coleta.title)

<title>
	SSP
</title>


In [6]:
tabelas = coleta.find_all('table', attrs={'class':'table table-striped table-hover table-condensed'})

In [7]:
print('Localizado:', len(tabelas),'tabelas')

Localizado: 121 tabelas


tratamentos dos dados

In [8]:
Rotulos = []
Dados = []
Periodos = []

Registro_Inicial = datetime.datetime(2021, 11, 22)

Loop = 0

for Consulta in tabelas:
  Linha = Consulta.find_all('tr')[2]
  Rotulo = Linha.find_all('td')[0].text
  Informação = Linha.find_all('span')[0].text
  Data = Registro_Inicial - relativedelta(months=Loop)

  #caso tenha feminicidio

  if Rotulo == 'FEMINICÍDIO':
    Rotulos.append(Rotulo)
    Dados.append(Informação)
    Periodos.append(Data)
  else:
   pass

    #somando o loop para dar o mes
  Loop = Loop + 1

#organização dos dados em um dicionario
Dicionario = {
    'Rotulo': Rotulos, 
    'Quantidade': Dados,
     'Periodo': Periodos
}
 
 #passando o dicionario como base de dados

Base_Feminicidio = pd.DataFrame(Dicionario)

In [9]:
Base_Feminicidio.head()

,Rotulo,Quantidade,Periodo
0,FEMINICÍDIO,6,2021-11-22
1,FEMINICÍDIO,8,2021-10-22
2,FEMINICÍDIO,8,2021-09-22
3,FEMINICÍDIO,7,2021-08-22
4,FEMINICÍDIO,26,2021-07-22


In [10]:
Base_Feminicidio.describe()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.



,Rotulo,Quantidade,Periodo
count,45,45,45
unique,1,19,45
top,FEMINICÍDIO,8,2019-11-22 00:00:00
freq,45,6,1
first,NaN,NaN,2018-03-22 00:00:00
last,NaN,NaN,2021-11-22 00:00:00


In [11]:
#preparando os dados para a serie
Base_Feminicidio = Base_Feminicidio.set_index('Periodo')
#ordenar a data
Base_Feminicidio = Base_Feminicidio.sort_values(by='Periodo')

In [12]:
Base_Feminicidio

,Rotulo,Quantidade
Periodo,,
2018-03-22,FEMINICÍDIO,5
2018-04-22,FEMINICÍDIO,8
2018-05-22,FEMINICÍDIO,8
2018-06-22,FEMINICÍDIO,14
2018-07-22,FEMINICÍDIO,16
2018-08-22,FEMINICÍDIO,6
2018-09-22,FEMINICÍDIO,16
2018-10-22,FEMINICÍDIO,11
2018-11-22,FEMINICÍDIO,11


In [13]:
#verificando os tipos de colunas
Base_Feminicidio.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 45 entries, 2018-03-22 to 2021-11-22
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Rotulo      45 non-null     object
 1   Quantidade  45 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB


In [14]:
#convertendo a coluna para inteiro
Base_Feminicidio['Quantidade'] = pd.to_numeric(Base_Feminicidio['Quantidade'])

In [15]:
#plotar o historico agora
figura = Dash.Figure()
figura.add_trace(Dash.Scatter(x = Base_Feminicidio.index, y = Base_Feminicidio.Quantidade,
                   mode='lines',
                   name='Quantidade',
                   marker_color= 'red',))

figura.update_layout(
    title='Historico de Feminicidio SP',
    titlefont_size=28,

    xaxis= dict(
        title ='Periodo Historico',
        titlefont_size=16,
        tickfont_size=14),


    height = 500,
    
  
    yaxis = dict(
        title='Quantidade',
        titlefont_size=16,
        tickfont_size=14),

    legend = dict(
        y =1, x = 1, 
        bgcolor ='red',
        bordercolor ='red'))


figura.show()
       



In [23]:
#Treinar o Modelo
from pmdarima.arima import auto_arima

In [ ]:
Funcao_auto_arima = auto_arima(
    #treinar o modelo
    Base_Feminicidio['Quantidade'],
    start_p=1, start_q=1,
    max_p=6, max_q=6,
    m=12 #diferenciação sazonal,
    start_P=0,
    seasonal=True,
    d=1,
    D=1,
    trace=True,
    error_action='ignore',
    stepwise=True)

In [25]:
# em construção!!!!